Таска 1

In [ ]:
def recall_at_k(target: list[int], predict: list[list[int]], k: int) -> float:

    if not target or not predict or len(target) != len(predict):
        raise ValueError("target and predict must be non-empty lists of the same length.")
    if k <= 0:
        raise ValueError("k must be a positive integer.")

    num_queries = len(target)
    recalls = 0
    for i in range(num_queries):
        correct_doc_id = target[i]
        predicted_docs_at_k = predict[i][:k]

        if correct_doc_id in predicted_docs_at_k:
            recalls += 1

    return recalls / num_queries if num_queries > 0 else 0.0

def mean_reciprocal_rank(target: list[int], predict: list[list[int]]) -> float:

    if not target or not predict or len(target) != len(predict):
        raise ValueError("target and predict must be non-empty lists of the same length.")

    num_queries = len(target)
    reciprocal_ranks = 0.0
    for i in range(num_queries):
        correct_doc_id = target[i]
        predicted_docs = predict[i]
        try:
            rank = predicted_docs.index(correct_doc_id) + 1
            reciprocal_ranks += 1.0 / rank
        except ValueError:
            reciprocal_ranks += 0.0

    return reciprocal_ranks / num_queries if num_queries > 0 else 0.0

target_ids = [1, 5, 2]
predicted_ids = [[10, 20, 1, 30, 40],
     [50, 5, 60, 70],
     [80, 90, 100]]
k = 3
recall_k = recall_at_k(target_ids, predicted_ids, k)
print(f"Recall@{k}: {recall_k}")
mrr_score = mean_reciprocal_rank(target_ids, predicted_ids)
print(f"MRR: {mrr_score}")

Recall@3: 0.6666666666666666
MRR: 0.27777777777777773


Таска 2

In [ ]:
! pip install datasets

In [ ]:
! pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import sys

print("Загрузка датасета sentence-transformers/natural-questions...")
dataset = load_dataset("sentence-transformers/natural-questions", split='train')
print("Датасет загружен.")
print(f"Загружен сплит с {len(dataset)} записями.")
print("Доступные колонки:", dataset.column_names)
print("Разделение датасета на train (80%) и test (20%)...")
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_data = split_dataset['train']
test_data = split_dataset['test']
print("Датасет разделен.")
print(f"Размер обучающей выборки: {len(train_data)}")
print(f"Размер тестовой выборки: {len(test_data)}")

try:
    train_documents = [item['answer'] for item in train_data]
    train_questions = [item['query'] for item in train_data]
    test_documents = [item['answer'] for item in test_data]
    test_questions = [item['query'] for item in test_data]
except KeyError as e:
    print(f"Ошибка: Ожидаемое поле {e} не найдено в датасете.")
    print("Пожалуйста, проверьте доступные колонки:")
    print(dataset.column_names)
    raise

test_pairs = list(zip(test_questions, test_documents))
print("Настройка TF-IDF векторизатора на обучающих документах...")
tfidf_vectorizer = TfidfVectorizer()
if train_documents:
    tfidf_vectorizer.fit(train_documents)
    print("TF-IDF векторизатор настроен.")
else:
    print("Ошибка: Обучающая выборка документов пуста. Невозможно настроить векторизатор.")
    raise ValueError("Обучающая выборка документов пуста.")

print("Векторизация вопросов и документов тестовой выборки...")
if test_questions and test_documents:
    test_question_vectors = tfidf_vectorizer.transform(test_questions)
    test_document_vectors = tfidf_vectorizer.transform(test_documents)
    print("Векторизация завершена.")
else:
    print("Ошибка: Тестовая выборка вопросов или документов пуста. Невозможно векторизовать.")
    raise ValueError("Тестовая выборка вопросов или документов пуста.")

print("Расчет косинусной близости и ранжирование...")
if test_question_vectors.shape[0] > 0 and test_document_vectors.shape[0] > 0:
    similarity_matrix = cosine_similarity(test_question_vectors, test_document_vectors)
    print("Расчет близости завершен.")
else:
    print("Ошибка: Векторы тестовой выборки пусты. Невозможно рассчитать близость.")
    raise ValueError("Векторы тестовой выборки пусты.")

print("Расчет метрик MRR и Recall@k...")
mrr_scores = []
recall_at_1 = 0
recall_at_3 = 0
recall_at_10 = 0
num_test_questions = len(test_questions)

if num_test_questions == 0:
    print("Ошибка: Количество тестовых вопросов равно нулю. Невозможно рассчитать метрики.")
else:
    for i in range(num_test_questions):
        true_document_index = i
        question_similarity_scores = similarity_matrix[i]
        ranked_document_indices = np.argsort(question_similarity_scores)[::-1]
        rank = -1
        indices_of_true_doc = np.where(ranked_document_indices == true_document_index)[0]

        if indices_of_true_doc.size > 0:
            rank = indices_of_true_doc[0] + 1
            mrr_scores.append(1.0 / rank)
            if rank <= 1:
                recall_at_1 += 1
            if rank <= 3:
                recall_at_3 += 1
            if rank <= 10:
                recall_at_10 += 1

    mean_mrr = np.mean(mrr_scores) if mrr_scores else 0
    final_recall_at_1 = recall_at_1 / num_test_questions if num_test_questions > 0 else 0
    final_recall_at_3 = recall_at_3 / num_test_questions if num_test_questions > 0 else 0
    final_recall_at_10 = recall_at_10 / num_test_questions if num_test_questions > 0 else 0


    print("\n--- Результаты TF-IDF Baseline ---")
    print(f"Количество тестовых вопросов: {num_test_questions}")
    print(f"MRR: {mean_mrr:.4f}")
    print(f"Recall@1: {final_recall_at_1:.4f}")
    print(f"Recall@3: {final_recall_at_3:.4f}")
    print(f"Recall@10: {final_recall_at_10:.4f}")

Загрузка датасета sentence-transformers/natural-questions...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Датасет загружен.
Загружен сплит с 100231 записями.
Доступные колонки: ['query', 'answer']
Разделение датасета на train (80%) и test (20%)...
Датасет разделен.
Размер обучающей выборки: 80184
Размер тестовой выборки: 20047
Настройка TF-IDF векторизатора на обучающих документах...
TF-IDF векторизатор настроен.
Векторизация вопросов и документов тестовой выборки...
Векторизация завершена.
Расчет косинусной близости и ранжирование...
Расчет близости завершен.
Расчет метрик MRR и Recall@k...

--- Результаты TF-IDF Baseline ---
Количество тестовых вопросов: 20047
MRR: 0.5385
Recall@1: 0.4102
Recall@3: 0.6145
Recall@10: 0.7846


**Метрики:**

MRR (Mean Reciprocal Rank): Показывает среднее обратное ранга первого релевантного документа. Если релевантный документ всегда находится на первой позиции, то MRR равен 1. Если он на второй позиции, MRR = 1/2. Среднее значение MRR говорит о том, насколько высоко в среднем ранжируется правильный ответ. Чем выше MRR (ближе к 1), тем лучше. Низкое значение указывает на то, что правильный документ часто оказывается далеко от первых позиций.

Recall@n: Эта метрика показывает долю запросов, для которых истинно релевантный документ был найден среди первых n отранжированных документов.

Recall@1: Процент запросов, где правильный документ оказался на первой позиции.

Recall@3: Процент запросов, где правильный документ оказался среди первых трех позиций.

Recall@10: Процент запросов, где правильный документ оказался среди первых десяти позиций.

Что можно сказать о метриках:

MRR: 0.5385

Recall@1: 0.4102

Recall@3: 0.6145

Recall@10: 0.7846

Значения Recall@k низкие, особенно Recall@1, это говорит о том, что TF-IDF не очень эффективно поднимает истинно релевантные документы на верхние позиции в списке результатов для многих запросов.
Значение MRR показывает среднюю "стоимость" поиска релевантного документа (насколько глубоко приходится смотреть). Низкий MRR также подтверждает, что правильные ответы часто оказываются низко.

2. Какие ограничения есть у текущего подхода (TF-IDF)?

Подход на основе TF-IDF, несмотря на свою простоту и эффективность для базовых задач, имеет ограничения:

- Не учитывает семантику (смысл слов);

- Проблема словарного запаса (Out-of-Vocabulary - OOV);

- Зависимость от точного совпадения терминов;

- Не учитывает порядок слов и структуру предложения;

- Масштабируемость с точки зрения размерности;

- Отсутствие учета важности слов в контексте запроса.

Таска 3

In [ ]:
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используется устройство: {device}")

print("Загрузка датасета sentence-transformers/natural-questions...")
dataset = load_dataset("sentence-transformers/natural-questions", split='train')
print("Датасет загружен.")
print(f"Загружен сплит с {len(dataset)} записями.")
print("Доступные колонки:", dataset.column_names)
print("Разделение датасета на train (80%) и test (20%)...")
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_data = split_dataset['train']
test_data = split_dataset['test']
print("Датасет разделен.")
print(f"Размер обучающей выборки: {len(train_data)}")
print(f"Размер тестовой выборки: {len(test_data)}")

try:
    i = train_data[0]['query']
    j = train_data[0]['answer']
    test_documents = [item['answer'] for item in test_data]
    test_questions = [item['query'] for item in test_data]

except KeyError as e:
    print(f"Ошибка: Ожидаемое поле {e} не найдено в датасете.")
    print("Пожалуйста, проверьте доступные колонки:")
    print(dataset.column_names)
    raise ValueError(f"Отсутствует необходимое поле в датасете: {e}")

if not test_questions or not test_documents:
     print("Ошибка: Тестовая выборка вопросов или документов пуста.")
     raise ValueError("Тестовая выборка пуста.")

model_name = "intfloat/multilingual-e5-base"
print(f"Загрузка модели {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
print("Модель загружена.")

def get_embeddings(texts, model, tokenizer, device, prefix=""):
    processed_texts = [prefix + t for t in texts]
    encoded_input = tokenizer(processed_texts, padding=True, truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        model_output = model(**encoded_input)
        embeddings = average_pool(model_output.last_hidden_state, encoded_input['attention_mask'])

    embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
    return embeddings.cpu().numpy()

def average_pool(last_hidden_states, attention_mask):
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[:, None]

print("Векторизация тестовых вопросов и документов с помощью E5...")
batch_size = 64
test_question_embeddings = []
for i in range(0, len(test_questions), batch_size):
    batch_questions = test_questions[i:i+batch_size]
    embeddings = get_embeddings(batch_questions, model, tokenizer, device, prefix="query: ")
    test_question_embeddings.append(embeddings)
test_question_embeddings = np.concatenate(test_question_embeddings, axis=0)

test_document_embeddings = []
for i in range(0, len(test_documents), batch_size):
    batch_documents = test_documents[i:i+batch_size]
    embeddings = get_embeddings(batch_documents, model, tokenizer, device, prefix="passage: ")
    test_document_embeddings.append(embeddings)
test_document_embeddings = np.concatenate(test_document_embeddings, axis=0)

print("Векторизация завершена.")
print(f"Форма эмбеддингов вопросов: {test_question_embeddings.shape}")
print(f"Форма эмбеддингов документов: {test_document_embeddings.shape}")

print("Расчет косинусной близости...")
similarity_matrix = cosine_similarity(test_question_embeddings, test_document_embeddings)
print("Расчет близости завершен.")

print("Расчет метрик MRR и Recall@k...")
mrr_scores = []
recall_at_1 = 0
recall_at_3 = 0
recall_at_10 = 0

num_test_questions = len(test_questions)
if num_test_questions == 0:
    print("Ошибка: Количество тестовых вопросов равно нулю. Невозможно рассчитать метрики.")
else:
    for i in range(num_test_questions):
        true_document_index = i
        question_similarity_scores = similarity_matrix[i]
        ranked_document_indices = np.argsort(question_similarity_scores)[::-1]
        rank = -1
        indices_of_true_doc = np.where(ranked_document_indices == true_document_index)[0]

        if indices_of_true_doc.size > 0:
            rank = indices_of_true_doc[0] + 1

            mrr_scores.append(1.0 / rank)
            if rank <= 1:
                recall_at_1 += 1
            if rank <= 3:
                recall_at_3 += 1
            if rank <= 10:
                recall_at_10 += 1

    mean_mrr = np.mean(mrr_scores) if mrr_scores else 0
    final_recall_at_1 = recall_at_1 / num_test_questions if num_test_questions > 0 else 0
    final_recall_at_3 = recall_at_3 / num_test_questions if num_test_questions > 0 else 0
    final_recall_at_10 = recall_at_10 / num_test_questions if num_test_questions > 0 else 0

    print("\n--- Результаты E5 Baseline ---")
    print(f"Количество тестовых вопросов: {num_test_questions}")
    print(f"MRR: {mean_mrr:.4f}")
    print(f"Recall@1: {final_recall_at_1:.4f}")
    print(f"Recall@3: {final_recall_at_3:.4f}")
    print(f"Recall@10: {final_recall_at_10:.4f}")

Используется устройство: cuda
Загрузка датасета sentence-transformers/natural-questions...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train-00000-of-00001.parquet:   0%|          | 0.00/44.0M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Модель загружена.
Векторизация тестовых вопросов и документов с помощью E5...
Векторизация завершена.
Форма эмбеддингов вопросов: (20047, 768)
Форма эмбеддингов документов: (20047, 768)
Расчет косинусной близости...
Расчет близости завершен.
Расчет метрик MRR и Recall@k...

--- Результаты E5 Baseline ---
Количество тестовых вопросов: 20047
MRR: 0.7997
Recall@1: 0.6941
Recall@3: 0.8904
Recall@10: 0.9686


1. Какие получились метрики? Что можно о них сказать?

MRR: 0.7997

Recall@1: 0.6941

Recall@3: 0.8904

Recall@10: 0.9686

Значения значительно выше. Высокий MRR и высокие Recall@k  указывают на то, что E5 лучше понимает семантику запросов (и документов) и ранжирует выше релевантные ответы.
2. Стало ли лучше в сравнении с TF-IDF? Почему?

Да, стало лучше. Модели, основанные на трансформерах, обучаются на огромных корпусах текста для понимания взаимосвязей между словами в различных контекстах. Они генерируют семантические эмбеддинги — векторы чисел, которые захватывают смысл текста.
Основные преимущества E5 перед TF-IDF в этой задаче:
- понимание семантики;
- учет контекста;
- обработка Out-of-Vocabulary (OOV);
- понимание структуры предложения.

В результате, когда мы сравниваем эмбеддинги вопроса и документа с помощью косинусной близости, мы фактически измеряем их семантическое сходство. Это делает E5 гораздо более мощным инструментом для задач поиска по смыслу.

-------------------------------------